In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import prologuefunctions as prologue #Library for loading data
from IPython.core.debugger import set_trace
from torch import optim
from torch import Tensor
import matplotlib.pyplot as plt

# Load Data

In [2]:
N = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)

Using downloaded and verified file: ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz
Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw
Using downloaded and verified file: ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw
Using downloaded and verified file: ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw
Using downloaded and verified file: ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz
Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw
Processing...
Done!


# Data Preprocess

In [3]:
train_input_centered = train_input - train_input.mean(0)
print(train_input.narrow(0,0,100).size())

torch.Size([100, 2, 14, 14])


# Constructing NN

In [44]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(2, 16, kernel_size = 3)
        self.bn1 = nn.BatchNorm2d(16)
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.fc1_aux = nn.Linear(128, 100)
        self.out1 = nn.Linear(100, 20)
        
        self.fc1 = nn.Sequential(nn.Dropout2d(p=0.4), nn.Linear(128,100))
        self.out2 = nn.Linear(100, 2)
        
    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = self.bn1(x)
        
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = self.bn2(x)
        
        y = F.relu(self.fc1_aux(x.view(-1, 128)))
        y = self.out1(y)
        y = y.view(-1,2,10)
        
        x = F.relu(self.fc1(x.view(-1,128)))
    
        x = self.out2(x)
        
        return x, y



In [38]:
class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size):
        super(ResBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(channels, channels, kernel_size, padding = (kernel_size - 1)//2 )
        self.bn1 = nn.BatchNorm(channels)
        
        self.conv2 = nn.Conv2d(channels, channels, kernel_size, padding = (kernel_size - 1)//2)
        self.bn2 = nn.BarchNorm(channels)
        
    def forward(self, x):
        
        y = self.conv1(x)
        y = self.bn1(y)
        
        y = self.conv2(y)
        y = self.bn2(y)
        
        y = y + x
        y = F.relu(y)
        
        return y
    
    

# Training NN

- Batch
- number epochs
- learning rate


In [39]:
def training_model(train_input, train_target, model, batch, lr):
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)
    criterion = nn.CrossEntropyLoss()
    total_loss = 0
    for b in range(0, train_input.size(0), batch):
        output = model(train_input.narrow(0, b, batch))            
        loss = criterion(output, train_target.narrow(0, b, batch))
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss

In [40]:
def compute_nb_errors(model, data_input, data_target, batch):
    
    nb_data_errors = 0

    for b in range(0, data_input.size(0), batch):
        output = model(data_input.narrow(0, b, batch))
        _, predicted_classes = torch.max(output, 1)
        for k in range(batch):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1
    return nb_data_errors

In [48]:
def training_aux(train_input, train_target, train_classes, model, batch, lr):
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)
    Binary_Criterion = nn.CrossEntropyLoss()
    Aux_Criterion = nn.CrossEntropyLoss()
    total_loss_aux = 0
    total_loss_bin = 0
    final_total_loss = 0
    for b in range(0, train_input.size(0), batch):
        output, aux = model(train_input.narrow(0, b, batch))
        target_classes = train_classes.narrow(0, b, batch)
        target_comparison = train_target.narrow(0, b, batch)
        aux_loss = Aux_Criterion(aux[:,0], target_classes[:,0]) + Aux_Criterion(aux[:,0], target_classes[:,1])
        binary_loss = Binary_Criterion(output, target_comparison)
        final_loss = 0.9*binary_loss + 0.1*aux_loss
        model.zero_grad()
        final_loss.backward()
        optimizer.step()
        total_loss_aux += aux_loss
        total_loss_bin += binary_loss
        final_total_loss += final_loss
    return final_total_loss, total_loss_aux, total_loss_bin
    
    

In [49]:
def compute_nb_errors_aux(model, data_input, data_target, batch):
    
    nb_data_errors = 0

    for b in range(0, data_input.size(0), batch):
        output, _ = model(data_input.narrow(0, b, batch))
        _, predicted_classes = torch.max(output, 1)
        for k in range(batch):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1
    return nb_data_errors

In [50]:
epochs = 100
losses = torch.zeros((3, epochs))
model = Net()
for e in range(epochs):
    training_aux(train_input, train_target, train_classes, model , 100, 0.1)
    print(compute_nb_errors_aux(model, train_input,train_target, 100), " ",compute_nb_errors_aux(model, test_input,test_target, 100))



269   285
227   265
209   251
172   240
146   235
128   221
121   234
115   229
91   217
85   225
83   221
73   218
76   214
60   228
45   237
48   213
67   232
46   210
51   201
38   208
39   216
37   202
27   226
57   214
35   219
27   223
35   221
23   216
21   216
38   220
24   220
39   204
30   206
28   199
19   214
13   213
14   219
25   227
21   216
14   208
16   210
31   210
19   198
17   229
27   207
11   201
23   224
8   202
19   229
21   208
10   205
22   196
21   189
11   197
12   210
18   223
15   213
17   211
10   201
17   208
12   209
11   191
10   203
10   215
12   197
9   210
10   204
8   214
10   197
17   221
17   210
12   210
9   214
12   211
10   195
10   197
9   205
9   215
14   207
12   226
14   198
12   213
8   200
9   195
11   210
13   189
9   209
19   187
20   216
11   202
9   201
10   211
8   207
12   214
7   213
6   207
10   203
5   197
14   197
11   199


tensor([[1, 2, 3],
        [4, 5, 6]])

In [61]:
train_classes.narrow(0,1,3)[:,0].view(3,1)

tensor([[5],
        [7],
        [9]])